In [1]:
from Lib.myModules.my_ML_Modules_1 import *
import math
import sys
import contextlib

In [2]:
data_dir = "../data"
infantNum = 8
ECG_dataset = f"{data_dir}/infant{infantNum}_ecg"
RESP_dataset = f"{data_dir}/infant{infantNum}_resp"
# Read all the data
signal_ecg_0 = wfdb.rdsamp(f"{data_dir}/infant{infantNum}_ecg")
signal_resp_0 = wfdb.rdsamp(f"{data_dir}/infant{infantNum}_resp")
print(f'ECG DATA: {signal_ecg_0[1]}')
print(F'RESP DATA: {signal_resp_0[1]}')
totalDuration_ECG = num2Time(num=signal_ecg_0[1]['sig_len'], fs=signal_ecg_0[1]['fs'])
print(f'{totalDuration_ECG} sec, {secs2minutes(totalDuration_ECG)} minutes, {secs2hours(totalDuration_ECG)} hours')
totalDuration_RESP = num2Time(num=signal_resp_0[1]['sig_len'], fs=signal_resp_0[1]['fs'])
print(f'{totalDuration_RESP} sec, {secs2minutes(totalDuration_RESP)} minutes, {secs2hours(totalDuration_RESP)} hours')

ECG DATA: {'fs': 500, 'sig_len': 44288000, 'n_sig': 1, 'base_date': None, 'base_time': None, 'units': ['mV'], 'sig_name': ['II'], 'comments': []}
RESP DATA: {'fs': 50, 'sig_len': 4428613, 'n_sig': 1, 'base_date': None, 'base_time': None, 'units': ['NU'], 'sig_name': ['RESP'], 'comments': []}
88576.0 sec, 1476.2666666666667 minutes, 24.604444444444443 hours
88572.26 sec, 1476.2043333333334 minutes, 24.603405555555554 hours


In [3]:
# initialize the training
start_time_secs = hours2secs(2)
end_time_secs = hours2secs(2.2)
lr_model, pr_model, svr_rbf_model = training_pipeline(start_time_secs=start_time_secs, 
                                                        end_time_secs=end_time_secs, 
                                                        signal_ecg_0=signal_ecg_0,
                                                        signal_resp_0=signal_resp_0,
                                                        ECG_dataset=ECG_dataset,
                                                        RESP_dataset=RESP_dataset)

start: 3600000, end: 3960000
start: 360000, end: 396000
90th percentile: 0.1900387193332935, 10th percentile: -0.18792717800736802, IQR: 0.37796589734066155
Identified outliers: 14394
Data Shape: (360000,)
90th percentile: 17.408305226508833, 10th percentile: 15.500599967367158, IQR: 1.9077052591416752
Identified outliers: 123
Data Shape: (36000,)
[[140.8406518    6.2299657 ]
 [  6.2299657  157.02812419]]
Pearsons correlation: 0.042
Spearmans correlation: 0.060
---------------------------------------------
--- Linear Regression ---
Coefficient of determination: 0.0016626258305800778
Intercept: [122.94702]
Coefficients: [[0.04299657]]
Mean absolute error: 8.05
Mean squared error: 158.69
Root mean squared error: 12.60
---------------------------------------------
--- Polynomial Regression ---
Coefficient of determination: 0.02689069414369938
Intercept: [120.66269]
Coefficients: [[ 2.8628380e-09  2.4314298e-07  2.6502498e-06  4.1854109e-05
  -1.6403213e-06  1.6031976e-08]]
Mean absolute e

In [4]:
step = 0.25
startHrs = 10
endHrs = 12
logFilePath = f"../logs/infant{infantNum}_step{step}_startHrs{startHrs}_endHrs{endHrs}.log"

with open(logFilePath, 'a') as f:
    with contextlib.redirect_stdout(f):
        for i in np.arange(startHrs, endHrs, step):
            print("-------------------------------------------------------------------------")
            print(f"Data bin index: {i:.3f}")
            start_time_secs = hours2secs(i)
            end_time_secs = hours2secs(i+step)
            lr_model, pr_model, svr_rbf_model = cascade_training_pipeline(lr_model=lr_model, 
                                                                            pr_model=pr_model, 
                                                                            svr_rbf_model=svr_rbf_model, 
                                                                            start_time_secs=start_time_secs, 
                                                                            end_time_secs=end_time_secs,
                                                                            signal_ecg_0=signal_ecg_0,
                                                                            signal_resp_0=signal_resp_0,
                                                                            ECG_dataset=ECG_dataset,
                                                                            RESP_dataset=RESP_dataset)

In [5]:
# save the model
for i in [lr_model, pr_model, svr_rbf_model]:
    filename = f"../models/infant_{infantNum}/{namestr(i, globals())[0]}_step{step}_startHrs{startHrs}_endHrs{endHrs}.sav"
    joblib.dump(i, filename)
